https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE130353

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tarfile

import pandas as pd

import kraft

In [ ]:
SETTING = kraft.json.read("setting.json")

In [ ]:
_x_samples = kraft.geo.get_gse(
    "GSE130353", SETTING["data_directory_path"], overwrite=SETTING["overwrite"]
)

In [ ]:
file_path = kraft.internet.download(
    "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE130353&format=file",
    SETTING["data_directory_path"],
    file_name="GSE130353_RAW.tar",
    overwrite=SETTING["overwrite"],
)

In [ ]:
samples = []

with tarfile.open(file_path) as io:

    for tarinfo in io:

        n_transcript = pd.read_csv(
            io.extractfile(tarinfo),
            compression="gzip",
            skiprows=1,
            sep="\t",
            index_col=0,
        ).iloc[:, 6]

        n_transcript.name = tarinfo.name.split(sep="_")[1]

        samples.append(n_transcript)

gene_x_sample = pd.DataFrame(data=samples).T

gene_x_sample.index = kraft.name_biology.name_genes(gene_x_sample.index.to_numpy())

gene_x_sample.index.name = "Gene"

gene_x_sample = kraft.feature_x_sample.collapse(gene_x_sample)

gene_x_sample